In [36]:
import math
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
from tqdm import tqdm, trange
from sklearn.metrics import precision_score, recall_score, f1_score

from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import get_linear_schedule_with_warmup

from transformers import BertForTokenClassification, AdamW
from seqeval.metrics import f1_score, accuracy_score
import numpy as np

# Refrences
# https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/

MAX_LEN = 100
bs = 32
epochs = 5
max_grad_norm = 1.0


### DATA Ecoding

In [37]:
# Load NER data
# train_data = pd.read_csv("/home/jovyan/work/data/label_data/newner_train_data.csv")
# train_data = train_data[:5000] // start with smaller portion of data
# test_data = pd.read_csv("/home/jovyan/work/data/label_data/newner_test_data.csv")



In [38]:
# load health data
train_data = pd.read_csv("/home/jovyan/work/data/health/lstm/train.csv")
test_data = pd.read_csv("/home/jovyan/work/data/health/lstm/test.csv")

In [39]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),
                                                           s["tag"].values.tolist())]
                                                           # s["tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [40]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):
        if isinstance(word, float) and math.isnan(word):
            word = "NaN"

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [41]:
getter_train = SentenceGetter(train_data)
getter_test = SentenceGetter(test_data)

sentences_train = [[word[0] for word in sentence] for sentence in getter_train.sentences]
sentences_test = [[word[0] for word in sentence] for sentence in getter_test.sentences]

In [42]:
labels_train = [[s[1] for s in sentence] for sentence in getter_train.sentences]
labels_test = [[s[1] for s in sentence] for sentence in getter_test.sentences]
print(labels_train[0])

['O', 'O', 'O', 'DATE', 'O', 'O', 'O', 'O', 'O', 'DATE', 'O', 'O', 'DATE', 'O', 'O', 'O', 'O', 'O', 'DATE', 'B-AGE']


In [43]:
tag_values = list(set(train_data["tag"].values))
tag_values.append("ENDPAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [44]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

# GPU??
# torch.cuda.get_device_name(0)

### Tokenize Sentences
Since the BERT tokenizer is based a Wordpiece tokenizer it will split tokens in subword tokens. For example ‘gunships’ will be split in the two tokens ‘guns’ and ‘##hips’. We have to deal with the issue of splitting our token-level labels to related subtokens.

In [45]:
# load the pretrain model "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [46]:
def tokenize(sentences, labels):
    tokenized_texts_labels = []

    for sent, labs in zip(sentences,labels):
        tokenized_sentence, tokenized_labels = tokenize_and_preserve_labels(sent, labs)
        tokenized_texts_labels.append((tokenized_sentence, tokenized_labels))
    return tokenized_texts_labels

In [47]:
tokenized_train = tokenize(sentences_train, labels_train);
tokenized_test = tokenize(sentences_test,labels_test)

In [48]:
tokenized_texts_train = [token_label_pair[0] for token_label_pair in tokenized_train]
tokenized_labels_train = [token_label_pair[1] for token_label_pair in tokenized_train]

tokenized_texts_test = [token_label_pair[0] for token_label_pair in tokenized_test]
tokenized_labels_test = [token_label_pair[1] for token_label_pair in tokenized_test]

In [49]:
type(tokenized_labels_train)

list

In [50]:
words_ids_train = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_train],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")


tags_ids_train = pad_sequences([[tag2idx.get(l) for l in lab] for lab in tokenized_labels_train],
                     maxlen=MAX_LEN, value=tag2idx["ENDPAD"], padding="post",
                     dtype="long", truncating="post")



In [51]:
words_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")


tags_ids_test = pad_sequences([[tag2idx.get(l) for l in lab] for lab in tokenized_labels_test],
                     maxlen=MAX_LEN, value=tag2idx["ENDPAD"], padding="post",
                     dtype="long", truncating="post")

The purpose of the attention masks is to indicate to the model which tokens are actual input tokens and which ones are padded tokens. Padded tokens are typically not attended to by the model during computation, so the attention mask helps the model focus on the relevant parts of the input sequence.

In [52]:
attention_masks_train = [[float(i != 0.0) for i in ii] for ii in words_ids_train]
attention_masks_test = [[float(i != 0.0) for i in ii] for ii in words_ids_test]

In [53]:
words_train = torch.tensor(words_ids_train)
tags_train = torch.tensor(tags_ids_train)


words_test = torch.tensor(words_ids_test)
tags_test = torch.tensor(tags_ids_test)

masks_train = torch.tensor(attention_masks_train)
masks_test = torch.tensor(attention_masks_test)


In [54]:
train_data = TensorDataset(words_train, masks_train, tags_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

test_data = TensorDataset(words_test, masks_test, tags_test)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)

### Initial the model

In [55]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [56]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [57]:
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


### Finetune the model. A few epochs should be enough. The paper suggest 3-4 epochs.

In [60]:
def evaluate_per_tag(predictions, true_labels, tag_values):
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "ENDPAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                  for l_i in l if tag_values[l_i] != "ENDPAD"]

    # Calculate precision, recall, and F1-score for each tag
    tag_precision = precision_score(valid_tags, pred_tags, average=None)
    tag_recall = recall_score(valid_tags, pred_tags, average=None)
    tag_f1score = f1_score([valid_tags], [pred_tags], average=None)

    for tag, precision, recall, f1 in zip(tag_values, tag_precision, tag_recall, tag_f1score):
        print(f"Tag: {tag}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-Score: {f1}")
        print()

### Training the model

In [ ]:
from tqdm import trange

loss_values, validation_loss_values = [], []
total_steps = len(train_dataloader) * epochs
progress_bar = trange(total_steps, desc="Training Progress")

for step in progress_bar:

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

   # Get the batch from the dataloader.
    batch = next(iter(train_dataloader))


    # add batch to gpu
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    # clear any previously calculated gradients before performing a backward pass
    model.zero_grad()
    # forward pass: return the loss (rather than the model output) due to provided labels
    outputs = model(b_input_ids, token_type_ids=None,
                    attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    # backward pass: calculate the gradients.
    loss.backward()
    # track train loss
    total_loss += loss.item()
    # Clip the norm of the gradient
    # This is to help prevent the "exploding gradients" problem.
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    # update parameters
    optimizer.step()
    # Update the learning rate.
    scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    # Evaluation per Epoch
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        #not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(test_dataloader)
    validation_loss_values.append(eval_loss)
    evaluate_per_tag(predictions, true_labels, tag_values)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                  for p_i, l_i in zip(p, l) if tag_values[l_i] != "ENDPAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "ENDPAD"]


    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score([pred_tags], [valid_tags])))


Training Progress:   0%|          | 0/1790 [00:00<?, ?it/s]

Average train loss: 0.00015080482933108367


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: IDNUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/s

Tag: I-PER
Precision: 0.6105769230769231
Recall: 0.7366589327146171
F1-Score: 0.6566233766233766

Tag: IDNUM
Precision: 0.5222166749875187
Recall: 0.47937671860678277
F1-Score: 0.7904999266969652

Tag: I-STATE
Precision: 0.9209621993127147
Recall: 0.7674684994272624
F1-Score: 0.0

Tag: B-HOSPITAL
Precision: 0.0
Recall: 0.0
F1-Score: 0.5185185185185185

Tag: I-HOSPITAL
Precision: 0.45865082922456296
Recall: 0.803021385128507
F1-Score: 0.42694663167104113

Tag: I-PROFESSION
Precision: 0.125
Recall: 0.005555555555555556
F1-Score: 0.5105127259313906

Tag: B-STATE
Precision: 1.0
Recall: 0.04065040650406504
F1-Score: 0.4444444444444445

Tag: B-AGE
Precision: 0.0
Recall: 0.0
F1-Score: 0.22286541244573083

Tag: USERNAME
Precision: 0.3157051282051282
Recall: 0.32669983416252074
F1-Score: 0.0

Tag: B-ORGANIZATION
Precision: 0.0
Recall: 0.0
F1-Score: 0.7210596026490066

Tag: COUNTRY
Precision: 0.9201900465518069
Recall: 0.9446716263487215
F1-Score: 0.0

Tag: B-STREET
Precision: 0.0
Recall: 0.0
F1

Training Progress:   0%|          | 1/1790 [20:23<608:14:12, 1223.95s/it]

...

Validation F1-Score: 0.6249575282578658
Average train loss: 0.0002463720947004563


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: IDNUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/s

Tag: I-PER
Precision: 0.6173664122137404
Recall: 0.7505800464037123
F1-Score: 0.6663217796171753

Tag: IDNUM
Precision: 0.4582723279648609
Recall: 0.5737855178735105
F1-Score: 0.7981530343007914

Tag: I-STATE
Precision: 0.8907510862818125
Recall: 0.8218785796105383
F1-Score: 0.0

Tag: B-HOSPITAL
Precision: 0.0
Recall: 0.0
F1-Score: 0.5060240963855422

Tag: I-HOSPITAL
Precision: 0.5113386978785662
Recall: 0.6856974690994703
F1-Score: 0.42181069958847733

Tag: I-PROFESSION
Precision: 0.275
Recall: 0.06111111111111111
F1-Score: 0.5159204784224988

Tag: B-STATE
Precision: 0.8666666666666667
Recall: 0.10569105691056911
F1-Score: 0.5274725274725274

Tag: B-AGE
Precision: 0.5714285714285714
Recall: 0.04060913705583756
F1-Score: 0.22702702702702704

Tag: USERNAME
Precision: 0.38588235294117645
Recall: 0.27197346600331673
F1-Score: 0.0

Tag: B-ORGANIZATION
Precision: 0.0
Recall: 0.0
F1-Score: 0.7461191787681524

Tag: COUNTRY
Precision: 0.9219222099659489
Recall: 0.9470857762230872
F1-Score: 0.0

Training Progress:   3%|▎         | 49/1790 [15:59:24<558:52:26, 1155.63s/it]

Validation F1-Score: 0.6294075532486129
Average train loss: 0.00015113955394848764


In [ ]:
directory = '/home/jovyan/work/data/output/BERT_model'

model.save_pretrained(directory)
tokenizer.save_pretrained(directory)

KeyboardInterrupt: 